In [1]:
import mmap
import time

import import_ipynb
from library import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torch_geometric.nn import NNConv, global_mean_pool, GraphUNet
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from torch.utils.tensorboard import SummaryWriter

import glob
import os
import random
import json
import sys

np.set_printoptions(threshold=sys.maxsize)

os.chdir("C:/Users/user/Desktop/suhwan/connection_test/python_agent")

folderList = glob.glob("history*")

pathName = "history" + str(len(folderList))

print(pathName)

os.mkdir(pathName)



writer = SummaryWriter(pathName)
%matplotlib inline

MMAPNAME_SERVER = "Global\\pythonToOmnet"
MMAPNAME_CLIENT = "Global\\omnetToPython"
MMAPNAME = "Global\\omnetSharedMemory"
BUFFSIZE = 200000

modelNum = 0
availableJobNum = 0
nodeNum = 0
jobWaitingLength = 0
adjacency = 0

importing Jupyter notebook from library.ipynb
importing Jupyter notebook from job.ipynb
importing Jupyter notebook from jobqueue.ipynb
importing Jupyter notebook from network.ipynb
importing Jupyter notebook from dataplane.ipynb


C:\Users\user\anaconda3\envs\omnetTest\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


history66


In [2]:
learning_rate   = 0.001
gamma           = 0.99
entropy_weight  = 0.01
'''
entropy_weight : loss함수에 entropy라는 걸 더해주는 테크닉에서의 weight입니다.
이 기법은 A3C 논문에 나와있으며, 쓰는 이유는 exploration을 촉진하기 위해서입니다.
각 액션을 취할 확률을 거의 균등하게끔 업데이트해줌으로써 다양한 액션을 해볼 기회가 많아집니다.
actor-critic에서 이 기법으로 인한 성능 개선이 매우 효과적인 것으로 알고있으므로, 적용해주시면 좋을 듯 합니다.
'''
lmbda         = 0.97
eps_clip      = 0.2
'''
위 두 인자는 PPO에서 씁니다.
'''

loss_coef = 0.5

node_feature_num = 100
queue_feature_num = 100
hidden_feature_num = 0

job_generate_rate = 0.003

train_cycle = 200
is_train = True
train_quitient = 0

if not is_train:
    train_quitient = train_cycle
    

# Memory Map 관련 함수

In [3]:
def returnParsedResult(string):
    temp = string.split('/')
    return temp

def returnFinedResult(strResult):
    finedResult = ""
    for char in strResult[1:]:
        if char != "!": finedResult += char
        else: break

    return finedResult

def returnCommandString(commandList):
    string = ""
    for command in commandList:
        temp = command.split(' ')
        string += "{} {}/".format(temp[0], temp[1])

    return string

def sendOmnetCommand(myCommand):
    myCommand = "p" + myCommand + "!"
    byteMessage = myCommand.encode()
    shm.write(byteMessage)

def getOmnetCommand(sendResponse):
    while True:
        shm.seek(0)
        byteResult = shm.readline()
        strResult = byteResult.decode() # mmap에 저장돼있는 명령 구문 저장 

        if strResult[0] == 'c': # omnet의 처음 명령이면
            finedResult = returnFinedResult(strResult)
            shm.seek(0)
            
            if sendResponse:
                sendOmnetCommand("ok")

            # shm.seek(0)
            # byteResult = shm.readline()
            # strResult = byteResult.decode()
            # finedResult = returnFinedResult(strResult)
            # print(finedResult)
            return finedResult


In [4]:
shmToOmnet = mmap.mmap(0, BUFFSIZE, MMAPNAME_SERVER)
shmFromOmnet = mmap.mmap(0, BUFFSIZE, MMAPNAME_CLIENT)

shm = mmap.mmap(0, BUFFSIZE, MMAPNAME)

if shm:
    while True:
        shm.seek(0)
        byteResult = shm.readline()
        strResult = byteResult.decode() # mmap에 저장돼있는 명령 구문 저장 
        
        if strResult.replace("\x00", '') != '': # 메모리가 처음 생성된게 아니라면
            if strResult[0] == 'c' and strResult[1] == 'f': # omnet의 처음 명령이면
                finedResult = returnFinedResult(strResult)
                networkInfo = json.loads(finedResult[1:])

                modelNum = int(networkInfo['modelNum'])
                availableJobNum = int(networkInfo['availableJobNum'])
                nodeNum = int(networkInfo['nodeNum'])
                jobWaitingLength = int(networkInfo['jobWaitingQueueLength'])
                adjacency = eval(networkInfo['adjacencyList'])

                node_feature_num = 2 * (modelNum * availableJobNum)
                queue_feature_num = (nodeNum + modelNum) * jobWaitingLength
                hidden_feature_num = 10*(node_feature_num + queue_feature_num)

                shm.seek(0)

                sendOmnetCommand("init") # 입력 끝나면 omnet에 전송

                print("네트워크 초기화 완료")

                print(f"노드 개수 : {nodeNum}")
                print(f"네트워크 최대 job 개수 : {availableJobNum}")
                print(f"job 대기 가능 개수 : {jobWaitingLength}")
                print(f"최대 subtask 개수 : {modelNum}")
                print(f"인접 리스트 : {adjacency}")
                print(f"node_feature_num : {node_feature_num}")
                print(f"queue_feature_num : {queue_feature_num}")

                break



        else: # 메모리가 처음 생성되었다면
            shm.seek(0)
            byteMessage = "pf!".encode()
            shm.write(byteMessage)
            print("첫 번째 연결, 메모리 초기화 완료")
                

else:
    print("공유 메모리 접근 불가")

첫 번째 연결, 메모리 초기화 완료
네트워크 초기화 완료
노드 개수 : 7
네트워크 최대 job 개수 : 20
job 대기 가능 개수 : 15
최대 subtask 개수 : 5
인접 리스트 : [[0, 3, 1, 3, 1, 4, 1, 5, 2, 3, 2, 4, 2, 5, 3, 4, 3, 6, 4, 5, 4, 6, 5, 6], [3, 0, 3, 1, 4, 1, 5, 1, 3, 2, 4, 2, 5, 2, 4, 3, 6, 3, 5, 4, 6, 4, 6, 5]]
node_feature_num : 200
queue_feature_num : 180


In [5]:
            # elif strResult[0] == 'c': # omnet의 명령이면
            #     print("====================================================================================")
            #     finedResult = returnFinedResult(strResult)
            #     #printParsedResult(finedResult) # 네트워크 정보 출력
            #     print(finedResult)
                
            #     shm.seek(0)

                
            #     sendOmnetCommand("hello") # 입력 끝나면 omnet에 전송

In [6]:
class actor_network(nn.Module):
    def __init__(self):
        super(actor_network, self).__init__()
        self.data = []

        self.step = 0

        self.pi_mlp1 = nn.Sequential(nn.Linear(1, node_feature_num * node_feature_num), nn.ReLU())
        self.pi_s_ecc = NNConv(node_feature_num, node_feature_num, self.pi_mlp1, aggr='mean')

        self.pi_graph_u_net = GraphUNet(node_feature_num, 10, node_feature_num, 3, 0.8)

        self.pi_backbone = nn.Sequential(
            nn.Linear(node_feature_num + queue_feature_num, hidden_feature_num),
            nn.Linear(hidden_feature_num, hidden_feature_num),
            nn.ReLU(),
        )

        # prob_fc : 각 액션에 대한 확률.
        self.pi_prob_fc = nn.Linear(hidden_feature_num, nodeNum + 1) # nodeNum + voidAction

        self.v_mlp1 = nn.Sequential(nn.Linear(1, node_feature_num * node_feature_num), nn.ReLU())
        self.v_s_ecc = NNConv(node_feature_num, node_feature_num, self.v_mlp1, aggr='mean')

        self.v_graph_u_net = GraphUNet(node_feature_num, 10, node_feature_num, 3, 0.8)

        self.v_backbone = nn.Sequential(
            nn.Linear(node_feature_num + queue_feature_num, hidden_feature_num),
            nn.Linear(hidden_feature_num, hidden_feature_num),
            nn.ReLU(),
        )

        # prob_fc : 각 액션에 대한 확률.
        self.v_value_fc = nn.Linear(hidden_feature_num, 1)
        
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
              
        
    # policy DNN
    def pi(self, state):
        data, job_waiting_feature = state # data = graph data
        node_feature, link_feature, adjacency = data.x, data.edge_attr, data.edge_index
        """
        node_feature = F.relu(self.conv1(node_feature, adjacency, link_feature))
        node_feature = F.relu(self.conv2(node_feature, adjacency, link_feature))
        #node_feature = F.relu(self.conv3(node_feature, adjacency, link_feature))
        readout = global_mean_pool(node_feature, data.batch) # 모든 노드의 feature를 평균내서 하나의 벡터로 만들어주기.
        """

        #node_feature = F.relu(self.pi_s_gcn(node_feature, adjacency, link_feature))
        
        node_feature = self.pi_graph_u_net(node_feature, adjacency)
        node_feature = F.relu(self.pi_s_ecc(node_feature, adjacency, link_feature))
        node_feature = F.relu(self.pi_s_ecc(node_feature, adjacency, link_feature))
        
        readout = global_mean_pool(node_feature, data.batch)

        #print("readout", readout.shape)
        #print("job_waiting_feature", job_waiting_feature.shape)
        concat = torch.cat([readout, job_waiting_feature], dim=1) # 여기에 job waiting 벡터 붙이기.

        # print(concat.shape)

        concat = F.normalize(concat) # normalize

        feature_extract = self.pi_backbone(concat)

        output = self.pi_prob_fc(feature_extract) 

        prob = F.softmax(output, dim=1)
        
        # 아래는 엔트로피 구하는 과정
        log_prob = F.log_softmax(output, dim=1)
        entropy = (log_prob * prob).sum(1, keepdim=True)
        
        return prob, entropy, output

    # advantage network
    def v(self, state):
        data, job_waiting_feature = state
        node_feature, link_feature, adjacency = data.x, data.edge_attr, data.edge_index
        """node_feature = F.relu(self.conv1(node_feature, adjacency, link_feature))
        node_feature = F.relu(self.conv2(node_feature, adjacency, link_feature))
        #node_feature = F.relu(self.conv3(node_feature, adjacency, link_feature))
        readout = global_mean_pool(node_feature, data.batch)
        
        # job waiting vector concat
        concat = torch.cat([readout, job_waiting_feature], dim=1)"""

        #node_feature = F.relu(self.v_s_gcn(node_feature, adjacency, link_feature))
        
        """
        node_feature = self.v_graph_u_net(node_feature, adjacency)
        node_feature = F.relu(self.v_s_ecc(node_feature, adjacency, link_feature))
        node_feature = F.relu(self.v_s_ecc(node_feature, adjacency, link_feature))
        
        readout = global_mean_pool(node_feature, data.batch)
        # print(readout.shape)
        # print(job_waiting_feature.shape)
        concat = torch.cat([readout, job_waiting_feature], dim=1) # 여기에 job waiting 벡터 붙이기.

        concat = F.normalize(concat) # normalize

        feature_extract = self.v_backbone(concat)
        """
        node_feature = self.pi_graph_u_net(node_feature, adjacency)
        node_feature = F.relu(self.pi_s_ecc(node_feature, adjacency, link_feature))
        node_feature = F.relu(self.pi_s_ecc(node_feature, adjacency, link_feature))
        
        readout = global_mean_pool(node_feature, data.batch)

        #print("readout", readout.shape)
        #print("job_waiting_feature", job_waiting_feature.shape)
        concat = torch.cat([readout, job_waiting_feature], dim=1) # 여기에 job waiting 벡터 붙이기.

        # print(concat.shape)

        concat = F.normalize(concat) # normalize

        feature_extract = self.pi_backbone(concat)

        
        value = self.v_value_fc(feature_extract) # 앞부분은 pi랑 공유해야 하고, concat -> value_fc를 거치는 것만 다름.
        return value
        
    def put_data(self, transition):
        self.data.append(transition)
        
    
    # make_batch, train_net은 맨 위에 코드 기반 링크와 거의 동일합니다.
        
    def make_batch(self):
        network_lst, job_waiting_lst, a_lst, r_lst, next_network_lst, next_job_waiting_lst, prob_a_lst, sojourn_time_lst = [], [], [], [], [], [], [], []
        entropy_lst = []
        for transition in self.data:
            network, job_waiting, a, r, nxt_network, nxt_job_waiting, prob_a, entropy, sojourn_time = transition
                
            r_lst.append([r/10.0])
            network_lst.append(network)
            job_waiting_lst.append(job_waiting.tolist())
            a_lst.append([a])
            next_network_lst.append(nxt_network)
            next_job_waiting_lst.append(nxt_job_waiting.tolist())
            prob_a_lst.append([prob_a])
            entropy_lst.append([entropy])
            sojourn_time_lst.append([sojourn_time * 10])
        
        # r_lst = np.array(r_lst)
        # r_lst = (r_lst - np.mean(r_lst)) / np.std(r_lst)
        # r_lst = r_lst.tolist()

        for r in r_lst:
            writer.add_scalar("Reward/train", r[0], self.step)
            self.step += 1

        # gnn sample을 배치단위로 inference하려면 이렇게 묶어줘야 함.
        network_loader = DataLoader(network_lst, batch_size=len(network_lst))
        next_network_loader = DataLoader(next_network_lst, batch_size=len(next_network_lst))
        network_batch = next(iter(network_loader))
        next_network_batch = next(iter(next_network_loader))

        # print(job_waiting_lst)
        
        # job_waiting = torch.tensor(np.array(job_waiting_lst), dtype=torch.float)
        job_waiting = torch.squeeze(torch.tensor(np.array(job_waiting_lst), dtype=torch.float), dim=1)
        a = torch.tensor(a_lst)
        r = torch.tensor(r_lst, dtype=torch.float)
        sojourn_time = torch.Tensor(sojourn_time_lst)
        next_job_waiting = torch.squeeze(torch.tensor(np.array(next_job_waiting_lst), dtype=torch.float), dim=1)
        prob_a = torch.tensor(prob_a_lst, dtype=torch.float)
        entropy = torch.tensor(entropy_lst, dtype=torch.float)
        
        self.data = []
        return network_batch, job_waiting, a, r, next_network_batch, next_job_waiting, prob_a, entropy, sojourn_time
    
    
    
    def train_net(self):
        network_batch, job_waiting, a, r, next_network_batch, next_job_waiting, prob_a, entropy, sojourn_time = self.make_batch()
        
        for i in range(3):
            td_target = r + (torch.tensor([gamma])**sojourn_time) * self.v([next_network_batch, next_job_waiting])
            delta = td_target - self.v([network_batch, job_waiting])
            delta = delta.detach().numpy()
            advantage_lst = []
            advantage = 0.0
            for i, delta_t in enumerate(delta[::-1]):
                advantage = (gamma**sojourn_time[i][0]) * lmbda * advantage + delta_t[0]
                advantage_lst.append([advantage])
            advantage_lst.reverse()
            advantage = torch.tensor(advantage_lst, dtype=torch.float)

            v_loss = F.smooth_l1_loss(self.v([network_batch, job_waiting]) , td_target.detach())
            #print("v_loss", v_loss)
            
            """
            self.optimizer.zero_grad()
            v_loss.mean().backward()
            self.optimizer.step()
            """

            pi, _, _ = self.pi([network_batch, job_waiting])
            pi_a = pi.gather(1,a)
            ratio = torch.exp(torch.log(pi_a) - torch.log(prob_a))  # a/b == exp(log(a)-log(b))

            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * advantage
            pi_loss = - torch.min(surr1, surr2) - entropy_weight * entropy + v_loss

            #pi_loss = -(torch.log(pi_a)) * advantage

            #print("pi_loss", pi_loss)

            #print("pi_loss", pi_loss)

            self.optimizer.zero_grad()
            pi_loss.mean().backward()
            self.optimizer.step()

In [7]:
def main():
    global adjacency
    model = actor_network()
    # model.load_state_dict(torch.load("./history30/model.pth")) # -1 , +1
    # model.load_state_dict(torch.load("./history64/model.pth")) # original
    reward_history = []
    v_history = []
    
    # network topology의 edges(GNN 예제 링크 참고)
    adjacency = torch.tensor(adjacency, dtype=torch.long)

    # node_state, link_state = env.get_state()  # 환경으로부터 실제 state를 관측해 옴(OMNeT++에서 얻어온 statistic로 대체되어야 함).
    # node_state = torch.tensor(node_state, dtype=torch.float)
    # link_state = torch.tensor(link_state, dtype=torch.float)
    # job_waiting_state = env.get_job_waiting_vector()

    # network_state = Data(x=node_state, edge_attr=link_state, edge_index=adjacency)

    # for each time step
    time = 0
    step = 0

    average_reward = 0
    average_reward_num = 0

    pre_state = 0

    temp_history = []

    isStop = False
    node_selected_num = [0 for i in range(nodeNum)]
    void_selected_num = 0
    while True:
        msg = getOmnetCommand(True)
        if msg == "action": # omnet의 메세지, state 받으면 됨
            #print("action")
            for i in range(3):
                state = getOmnetCommand(False)

            #print(state)
            state = json.loads(state) # state 받았으므로 action 하면됨.

            node_waiting_state = np.array(eval(state['nodeState']))
            node_processing_state = np.array(eval(state['nodeProcessing']))
            link_state = np.array(eval(state['linkWaiting']))
            job_waiting_state = np.array(eval(state['jobWaiting']))
            activated_job_list = eval(state['activatedJobList'])
            isAction = int(state['isAction'])
            reward = float(state['reward'])
            averageLatency = float(state['averageLatency'])
            completeJobNum = int(state['completeJobNum'])
            sojournTime = float(state['sojournTime'])

            if averageLatency != -1:
                writer.add_scalar("averageLatency/train", averageLatency ,step)

            writer.add_scalar("completeJobNum/train", completeJobNum ,step)

            # 이 timestep에서 발생한 모든 샘플에 똑같은 보상 적용.
            for history in temp_history:
                history[3] = reward
                history[8] = sojournTime
                model.put_data(history)

            temp_history = []

            job_index = int(state['jobIndex'])

            print('sojourn time :', sojournTime)


            node_state = np.concatenate((node_waiting_state,node_processing_state) ,axis = 1)
            node_state = torch.tensor(node_state, dtype=torch.float)

            print(reward)

            link_state = torch.tensor(link_state, dtype=torch.float)

            job_waiting_num = 0
            job_waiting_queue = collections.deque()
            for job in job_waiting_state:
                if any(job): # 하나라도 0이 아닌 것 이 있으면 job이 있는것임.
                    job_waiting_num += 1
                    job_waiting_queue.append(job)
            
            job_waiting_state = torch.tensor(job_waiting_state, dtype=torch.float).view(1, -1)
            # print(job_waiting_state)

            network_state = Data(x=node_state, edge_attr=link_state, edge_index=adjacency)
            
            # print(job_waiting_queue)
            if job_waiting_num == 0:
                isAction = False
                

            if isAction:
                job_idx = job_index
                job = job_waiting_queue.popleft()
                src = -1
                dst = -1
                for i in range(nodeNum):
                    if job[i] == -1:
                        src = i
                    if job[i] == 1:
                        dst = i

                if src == -1:
                    src = dst
                    
                #print(f"src : {src}, dst : {dst}")
                #print(job)
                subtasks = job[nodeNum:]
                offloading_vector = []
                temp_data = []
                scheduling_start = False
                # print(subtasks)
                for order in range(len(subtasks)):
                    if subtasks[order] == 0:
                        break

                    network_state = Data(x=node_state, edge_attr=link_state, edge_index=adjacency)
                    # print(node_state.shape)
                    # print(link_state.shape)
                    # print(adjacency.shape)
                    # print(job_waiting_state.shape)
                    prob, entropy, output = model.pi([network_state, job_waiting_state])
                    
                    #print(f'prob : {prob}')
                    
                    m = Categorical(prob)
                    node = m.sample().item()
                    #print(f'node : {node}')
                    
                    # void action 실험용
                    # node = nodeNum 
                    
                    
                    # void action 뽑으면
                    if node == nodeNum and not scheduling_start: 
                        print("stop action")
                        prob[0] = torch.Tensor([0] * nodeNum + [1.0])
                        temp_history.append([network_state, job_waiting_state, node, 0, network_state, job_waiting_state, prob[0][node].item(), entropy, 0])
                        sendOmnetCommand("void")
                        void_selected_num += 1
                        
                        #print(reward)
                        per_timestep_reward = reward
                        if average_reward_num == 0:
                            average_reward = reward
                            average_reward_num = 1
                        else:
                            average_reward = average_reward + (reward - average_reward)/(average_reward_num + 1)
                            average_reward_num += 1

                        if step > 1:

                            #print(len(env.backlog))

                            for i in range(nodeNum):
                                node_tag = "node/" + str(i) + "/train"

                                writer.add_scalar(node_tag, node_selected_num[i], step)

                                
                            node_selected_num = [0 for i in range(nodeNum)]


                            if per_timestep_reward != 0:
                                # writer.add_scalar("Reward/train", per_timestep_reward,step)
                                writer.add_scalar("Value/train", model.v([network_state, job_waiting_state])[0], step)
                            

                            writer.flush()

                        if len(model.data) > 0 and step % train_cycle == train_quitient:
                            print("training....")
                            model.train_net()
                            modelPathName = pathName + "/model.pth"
                            torch.save(model.state_dict(), modelPathName)
                            writer.add_scalar("AverageReward/train", average_reward, step)
                            average_reward = 0
                            average_reward_num = 0   
                            
                        
                        node_selected_num = [0 for i in range(nodeNum)]
                        break

                    else:
                        scheduling_start = True

                    if scheduling_start:
                        # print(output)
                        # print(output[0])
                        # print(output[0][:nodeNum])
                        # print(F.softmax(output[0][:nodeNum], dim=0).tolist())
                        prob = torch.Tensor([F.softmax(output[0][:nodeNum], dim=0).tolist()])
                        # print(prob)
                        # print(torch.tensor([0]))
                        prob = torch.cat([prob[0], torch.tensor([0])]) # void action masking
                        # print(prob)
                        prob = torch.Tensor([prob.tolist()])


                        m = Categorical(prob[0])
                        node = m.sample().item()
                        

                        offloading_vector.append(node)

                        node_selected_num[node] += 1
                        
                        # state transition(환경으로부터 매번 state를 업데이트하는게 아닌, 현재 state를 기반으로 action에 해당하는 waiting만 더해줌).
                        # 아래의 구체적인 코드는 이해하시기 보단 OMNeT++에서 받아온 데이터로 새로 짜시는게 빠를 것 같습니다.
                        next_node_state = node_state.clone().detach()
                        next_job_waiting_state = job_waiting_state.clone().detach()
                        # print(next_job_waiting_state)
                        
                        next_node_state[node][modelNum * job_idx + order] += (subtasks[order]/100) # 100으로 나누는 이유 : 이렇게 해야 액션이 한쪽 노드로 쏠리게끔 학습이 되는 것을 어느정도 방지할 수 있는 것을 확인.
                        # 100으로 안나눠주면 너무 큰 값이 state에 추가되어서 inference시 가중치랑 곱해지면서 액션이 한쪽으로 확 쏠리는 걸로 예상됨.
                        next_network_state = Data(x=next_node_state, edge_attr=link_state, edge_index=adjacency)
                        next_job_waiting_state[0][nodeNum + order] = 0
                        
                        # model.put_data([network_state, job_waiting_state, node, 0, next_network_state, next_job_waiting_state, prob[0][node].item(), entropy])
                        
                        node_state = next_node_state
                        job_waiting_state = next_job_waiting_state

                        temp_history.append([network_state, job_waiting_state, node, 0, network_state, job_waiting_state, prob[0][node].item(), entropy, 0])
                
                if len(offloading_vector) != 0: # for문을 다 돌면 -> void action 안뽑으면
                    print(offloading_vector)
                    msg = str(offloading_vector)
                    sendOmnetCommand(msg)


            # 어쩔 수 없이 void action 해야되면
            else:
                #print("full")
                sendOmnetCommand("ok")
                prob, entropy, output = model.pi([network_state, job_waiting_state])

                for i in range(nodeNum):
                    output[0][i] = 0

                prob[0][nodeNum] = 1

                for history in temp_history:
                    history[3] = reward
                    history[8] = sojournTime
                    model.put_data(history)
                
                temp_history = []

                temp_history.append([network_state, job_waiting_state, nodeNum, 0, network_state, job_waiting_state, 1.0, entropy, 0])

                step += 1

                #print(reward)
                per_timestep_reward = reward
                if average_reward_num == 0:
                    average_reward = reward
                    average_reward_num = 1
                else:
                    average_reward = average_reward + (reward - average_reward)/(average_reward_num + 1)
                    average_reward_num += 1

                if step > 1:

                    #print(len(env.backlog))

                    for i in range(nodeNum):
                        node_tag = "node/" + str(i) + "/train"

                        writer.add_scalar(node_tag, node_selected_num[i], step)

                        
                    node_selected_num = [0 for i in range(nodeNum)]


                    if per_timestep_reward != 0:
                        # writer.add_scalar("Reward/train", per_timestep_reward,step)
                        writer.add_scalar("Value/train", model.v([network_state, job_waiting_state])[0], step)
                    

                    writer.flush()

                if len(model.data) > 0 and step % train_cycle == train_quitient:
                    print("training....")
                    model.train_net()
                    modelPathName = pathName + "/model.pth"
                    torch.save(model.state_dict(), modelPathName)
                    writer.add_scalar("AverageReward/train", average_reward, step)
                    average_reward = 0
                    average_reward_num = 0
                    
                    
                
                node_selected_num = [0 for i in range(nodeNum)]

        elif msg == "stop":
            
            step += 1
            sendOmnetCommand("ok")
            #print(reward)
            per_timestep_reward = reward
            if average_reward_num == 0:
                average_reward = reward
                average_reward_num = 1
            else:
                average_reward = average_reward + (reward - average_reward)/(average_reward_num + 1)
                average_reward_num += 1

            if step > 1:

                #print(len(env.backlog))

                for i in range(nodeNum):
                    node_tag = "node/" + str(i) + "/train"

                    writer.add_scalar(node_tag, node_selected_num[i], step)

                    
                node_selected_num = [0 for i in range(nodeNum)]


                # writer.add_scalar("Reward/train", per_timestep_reward,step)
                writer.add_scalar("Value/train", model.v([network_state, job_waiting_state])[0], step)

                writer.add_scalar("node/void/train", void_selected_num,step)
                void_selected_num = 0
                

                writer.flush()

            if len(model.data) > 0 and step % train_cycle == train_quitient:
                print("training....")
                model.train_net()
                modelPathName = pathName + "/model.pth"
                torch.save(model.state_dict(), modelPathName)
                writer.add_scalar("AverageReward/train", average_reward, step)
                average_reward = 0
                average_reward_num = 0
                
                
            
            node_selected_num = [0 for i in range(nodeNum)]

            

        """
        노드 state o
        링크 state o
        job waiting queue o
        job waiting num o
        activated job num o
        activated job list o
        """
        
        

if __name__ == '__main__':
    main()

sojourn time : 1.0
0.0
[2, 2]
sojourn time : 0.0
0.0
[6, 5, 3, 5]
sojourn time : 0.0
0.0
[5, 6, 0, 6]
sojourn time : 1.0
0.0
[2, 0, 5, 3]
sojourn time : 0.0
0.0
[2, 0, 3]
sojourn time : 0.0
0.0
[6, 6]
sojourn time : 0.0
0.0
[1, 0, 1, 0]
sojourn time : 1.0
0.0
[3, 5, 6]
sojourn time : 0.0
0.0
[2, 2, 5]
sojourn time : 0.0
0.0
[6, 4, 5]
sojourn time : 0.0
0.0
[1, 2]
sojourn time : 1.0
0.0
[1, 4, 5]
sojourn time : 0.0
0.0
[5, 4, 3, 3]
sojourn time : 0.0
0.0
[4]
sojourn time : 0.0
0.0
[3, 3, 6, 1, 4]
sojourn time : 0.0
0.0
[4, 2, 6]
sojourn time : 1.0
0.0
[5, 6, 5, 5]
sojourn time : 0.0
0.0
[6, 5, 0, 6, 0]
sojourn time : 0.0
0.0
[5]
sojourn time : 0.0
0.0
[1]
sojourn time : 4.90574
0.0
[6, 6, 2]
sojourn time : 0.281942
0.0
[1, 3]
sojourn time : 0.115144
33.865154
stop action
sojourn time : 0.1
0.0
[4, 6, 4, 3, 2]
sojourn time : 0.305157
0.0
[4]
sojourn time : 0.235204
0.0
[5, 1, 5, 6, 3]
sojourn time : 1.05454
0.0
[4, 3]
sojourn time : 0.913877
229.485991
[6, 6, 5, 0, 3]
sojourn time : 0.27

: 

: 

In [ ]:
output = torch.Tensor([[-0.0256,  0.0438,  0.0015, -0.0308, -0.0386, -0.0370, -0.0170, -0.0413]])
print(output[0][:nodeNum])


In [ ]:
a = [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [17.941176], [17.941176], [17.941176], [17.941176], [17.941176], [43.333333], [43.333333], [43.333333], [43.333333], [55.333333], [55.333333], [55.333333], [55.333333], [55.333333], [55.333333], [55.333333], [55.333333], [55.333333], [10.0], [10.0], [10.0], [10.0], [10.0], [26.25], [26.25], [26.25], [26.25], [24.054054], [24.054054], [24.054054], [24.054054], [24.054054], [14.137931], [14.137931], [41.5], [41.5], [21.538462], [21.538462], [37.058824], [21.315789], [21.315789], [21.315789], [21.315789], [24.571429], [24.571429], [24.571429], [24.571429], [32.307692], [32.307692], [32.307692], [32.307692], [7.592593], [7.592593], [7.592593], [7.592593], [7.592593], [14.705882], [112.5], [21.666667], [21.666667], [21.666667], [21.666667], [21.666667], [19.574468], [19.574468], [19.574468], [19.574468], [19.574468], [68.0], [68.0], [68.0], [17.692308], [17.692308], [17.692308], [43.636364], [43.636364], [43.636364], [43.636364], [1.52381], [1.52381], [1.52381], [1.52381], [1.52381], [2.966102], [35.333333], [35.333333], [3.608247], [3.608247], [3.608247], [3.608247], [3.608247], [17.037037], [17.037037], [17.037037], [10.0], [10.0], [10.0], [34.074074], [34.074074], [34.074074], [34.074074], [10.0], [87.142857], [77.272727], [10.09901], [10.09901], [70.833333], [70.833333], [70.833333], [70.833333], [59.375], [59.375], [59.375], [59.375], [33.214286], [33.214286], [33.214286], [46.0], [46.0], [46.0], [46.0], [20.357143], [20.357143], [20.357143], [20.357143], [28.4375], [22.857143], [22.857143], [22.857143], [22.857143], [56.923077], [56.923077], [56.923077], [56.923077], [56.923077], [16.842105], [18.666667], [18.666667], [18.666667], [18.666667], [18.666667], [5.806452], [5.806452], [5.806452], [35.0], [35.0], [35.0], [35.0], [32.352941], [32.352941], [10.0], [10.0], [16.666667], [16.666667], [16.666667], [16.666667], [34.375], [34.375], [34.375], [34.375], [34.375], [35.172414], [35.172414], [35.172414], [14.0], [14.0], [14.0], [8.695652], [18.4], [16.153846], [16.153846], [16.153846], [16.153846], [16.153846], [27.575758], [27.575758], [14.761905], [14.761905], [14.761905], [22.1875], [22.1875], [59.333333], [10.0], [10.0], [10.0], [27.666667], [20.0], [20.0], [20.0], [118.0], [118.0], [118.0], [118.0], [10.0], [10.0], [19.677419], [19.677419], [19.677419], [19.677419], [40.769231], [40.769231], [40.769231], [19.393939], [19.393939], [19.393939], [19.393939], [7.173913], [7.173913], [7.173913], [31.052632], [31.052632], [10.0], [25.714286], [25.714286], [25.714286], [25.714286], [28.888889], [40.526316], [31.363636], [31.363636], [31.363636], [31.363636], [21.5], [21.5], [21.5], [92.857143], [92.857143], [92.857143], [8.148148], [10.0], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [83.994253], [83.994253], [83.994253], [83.994253], [83.994253], [83.994253], [3.873874], [3.873874], [3.873874], [3.873874], [5.416667], [5.416667], [5.416667], [9.904762], [9.904762], [9.904762], [9.904762], [9.904762], [34.0], [26.956522], [26.956522], [26.956522], [3.714286], [3.714286], [3.714286], [3.714286], [7.037037], [7.037037], [7.037037], [7.037037], [24.615385], [16.25], [16.25], [27.142857], [27.142857], [4.736842], [4.736842], [4.736842], [10.0], [23.0], [23.0], [34.583333], [34.583333], [10.0], [10.0], [34.705882], [19.02439], [19.02439], [19.02439], [19.02439], [19.02439], [40.0], [40.0], [40.0], [44.444444], [44.444444], [44.444444], [44.444444], [44.444444], [20.416667], [20.416667], [20.416667], [5.30303], [5.30303], [5.30303], [6.666667], [6.666667]]

a = np.array(a)

print(sum(a))
print(np.std(a))
print(np.mean(a))
a = (a-np.mean(a)) / np.std(a)
a = a.tolist()
print(a)

In [ ]:
a = torch.Tensor([1, 2, 3, 4, 5])
b = torch.Tensor([[1], [2], [3], [4], [5]])

a = torch.Tensor([3])

print(a**b)